# Monte Carlo 


## 数值积分

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Define the Hamiltonian function
def hamiltonian(x):
    return np.sum(x**2, axis=1)  # Example: harmonic oscillator

# Define the Gibbs distribution
def gibbs_distribution(x, beta):
    return np.exp(-beta * hamiltonian(x))

# Monte Carlo integration
def monte_carlo_integration(f, beta, n_samples, dim):
    samples = np.random.uniform(-1, 1, size=(n_samples, dim))
    weights = gibbs_distribution(samples, beta)
    normalization = np.mean(weights)
    integral = np.mean(f(samples) * weights) / normalization
    return integral

# Example function to integrate
def example_function(x):
    return np.sum(x**2, axis=1)

# Parameters
beta = 1.0  # Inverse temperature
n_samples = 10000  # Number of Monte Carlo samples
dim = 6  # Dimensionality of the system

# Perform Monte Carlo integration
result = monte_carlo_integration(example_function, beta, n_samples, dim)
print(f"Monte Carlo integration result: {result}")

Monte Carlo integration result: 1.509700301961403


In [2]:
# Based on the trapezoidal method, re-internship the integration implemented by the Monte Carlo method, and compare the results
from scipy.integrate import trapezoid

# Trapezoidal rule integration
def trapezoidal_integration(f, beta, n_points_per_dim, dim):
    # Define the grid for one dimension
    points = np.linspace(-1, 1, n_points_per_dim)
    
    # Create a meshgrid for `dim` dimensions
    # This creates a list of `dim` arrays, where each array contains the coordinates for one dimension
    grid_coords = np.meshgrid(*[points]*dim, indexing='ij')
    
    # Reshape the grid coordinates into a (n_points^dim, dim) array
    # Each row is a point in the high-dimensional space
    grid_points = np.stack(grid_coords, axis=-1).reshape(-1, dim)
    
    # Evaluate the functions on the grid
    f_values = f(grid_points)
    weights = gibbs_distribution(grid_points, beta)
    
    # Reshape the values back to the grid shape for integration
    integrand_values = (f_values * weights).reshape([n_points_per_dim]*dim)
    weight_values = weights.reshape([n_points_per_dim]*dim)
    
    # Iteratively apply the trapezoidal rule over each dimension
    integral_numerator = integrand_values
    integral_denominator = weight_values
    for i in range(dim):
        integral_numerator = trapezoid(integral_numerator, points, axis=0)
        integral_denominator = trapezoid(integral_denominator, points, axis=0)
        
    return integral_numerator / integral_denominator

# Parameters for trapezoidal integration
# Note: The total number of points is n_points_per_dim^dim. 
# Be careful with high dimensionality. For dim=6, 10 points per dim is 1,000,000 total points.
n_points_per_dim = 10 

# Perform Trapezoidal integration
result_trapz = trapezoidal_integration(example_function, beta, n_points_per_dim, dim)

print(f"Monte Carlo integration result: {result}")
print(f"Trapezoidal rule integration result: {result_trapz}")


Monte Carlo integration result: 1.509700301961403
Trapezoidal rule integration result: 1.5283898681467503


## 随机变量的生成